In [1]:
import numpy as np 
import pandas as pd
import networkx as nx
import hypernetx as hnx
import matplotlib.pyplot as plt

<h5>Constructing the hypergraph of core nodes from the Enron email dataset. This code is taken from the other notebook.</h5>

In [2]:
file = open('data/addresses-email-Enron.txt')
map = {}

for pair in file:
    (k,v) = pair.split(" ")
    map[k] = v[:-1]    # [:-1] removes the '\n'
file.close()
print(f"There are {len(map)} core nodes")

file = open('data/email-Enron.txt')
set_list = []

for line in file:
    s = set() 
    for number in line[:-1].split(" "):
        if number in map:
            s.add(map[number])        
    if len(s) > 1: set_list.append(s)
file.close()

print(f"There are {len(set_list)} hyperedges of only core nodes")

There are 148 core nodes
There are 4058 hyperedges of only core nodes


<H2>Vertex based distance metrics</H2>

<h4>Hamming distance</h4>

This measures the difference between two hypergraphs by comparing the presence or absence of edges (hyperedges) between the same sets of vertices. It's akin to the traditional Hamming distance used for simple graphs but extended to hyperedges.

Simple terms: it is (AUB)' where A&B are hyperghraphs

In [3]:
# to demonstrate on the Enron hypergraph, I am going to break the hypergraph into to 2 hypergraphs. Each HG has 2029.
hg1 = set_list[:len(set_list)]
hg2 = set_list[len(set_list):]

hg_set1 = set(frozenset(s) for s in hg1)
hg_set2 = set(frozenset(s) for s in hg2)

print(f"The hamming distance is {len(hg_set1^hg_set2)}")

The hamming distance is 693


<H2>Hyperedge based distance metrics</H2>

<h4>Jaccard Distance</h4> 

This measures the dissimilarity between two hypergraphs based on their hyperedge sets. It is defined as 1−(∣E1∩E2∣)/(∣E1∪E2∣), where 𝐸1 and 𝐸2 are the sets of hyperedges in the two hypergraphs. The distance is 0 if the hypergraphs are identical and 1 if they share no common hyperedges.

Simple terms: it is the % of difference between hypergraphs' edges

In [4]:
hg1 = set_list[:len(set_list)]
hg2 = set_list[len(set_list):]

hg_set1 = set(frozenset(s) for s in hg1)
hg_set2 = set(frozenset(s) for s in hg2)

jd = 1 - (len(hg_set1 & hg_set2)/len(hg_set1 | hg_set2))

print(f"The jaccard distance is {jd}")

The jaccard distance is 1.0


In [5]:
'''
a better example 
'''

hg1 = [{1, 2, 3}, {3, 4}, {5}]
hg2 = [{1, 2, 3}, {3, 5}, {5, 6}]

hg_set1 = set(frozenset(s) for s in hg1)
hg_set2 = set(frozenset(s) for s in hg2)

jd = 1 - (len(hg_set1 & hg_set2)/len(hg_set1 | hg_set2))  # 1- (AUB)/(A∩B)

print(f"The jaccard distance is {jd}")


The jaccard distance is 0.8


<H2>Topological based distance metrics</H2>

This metric leverages the betweenness centrality of vertices within the hypergraphs. It measures how changes in vertex betweenness centrality affect the overall structure, providing a distance based on topological changes.

In [6]:
'''
The hypergraph needs to be reconstructed with ints
'''

file = open('data/addresses-email-Enron.txt')
map = {}

for pair in file:
    (k,v) = pair.split(" ")
    map[k] = v[:-1]    # [:-1] removes the '\n'
file.close()

file = open('data/email-Enron.txt')
set_list = []

for line in file:
    s = set() 
    for number in line[:-1].split(" "):
        if number in map:
            s.add(int(number))        
    if len(s) > 1: set_list.append(s)
file.close()

print(set_list[0])

{3, 116}


In [7]:
hg1 = set_list[:len(set_list)]
hg2 = set_list[len(set_list):]

G1 = nx.DiGraph()
G2 = nx.DiGraph()

for hyperedge in hg1:
    G1.add_nodes_from(hyperedge)
    for u in hyperedge:
        for v in hyperedge:
            if u != v:
                G1.add_edge(u, v)

for hyperedge in hg2:
    G2.add_nodes_from(hyperedge)
    for u in hyperedge:
        for v in hyperedge:
            if u != v:
                G2.add_edge(u, v)

centrality1 = nx.betweenness_centrality(G1)
centrality2 = nx.betweenness_centrality(G2)

diff = sum(abs(centrality1[node] - centrality2[node]) for node in G1.nodes & G2.nodes) # Limitation? It can only compute on the union

print(f"The topological distance is {diff}")

The topological distance is 0


In [8]:
hg1 = [{1, 2, 3}, {3, 4}, {5}]
hg2 = [{1, 2, 3}, {3, 5}, {5, 6}]

G1 = nx.DiGraph()
G2 = nx.DiGraph()

for hyperedge in hg1:
    G1.add_nodes_from(hyperedge)
    for u in hyperedge:
        for v in hyperedge:
            if u != v:
                G1.add_edge(u, v)

for hyperedge in hg2:
    G2.add_nodes_from(hyperedge)
    for u in hyperedge:
        for v in hyperedge:
            if u != v:
                G2.add_edge(u, v)

centrality1 = nx.betweenness_centrality(G1)
centrality2 = nx.betweenness_centrality(G2)

diff = sum(abs(centrality1[node] - centrality2[node]) for node in G1.nodes & G2.nodes)

print(f"The topological distance is {diff:.2f}")

The topological distance is 0.83
